<B>First Steps - Random Forrest Script - https://www.kaggle.com/c/street-view-getting-started-with-julia/details/julia-tutorial</B>

In [7]:
using Images, DataFrames, Colors,  HDF5, DecisionTree  

wd="/media/u01/analytics/scoring/k/FirstSteps/";   

#typeData could be either "train" or "test. labelsInfo should contain the IDs of each image to be read
#The images in the trainResized and testResized data files are 20x20 pixels, so imageSize is set to 400.
#path should be set to the location of the data files.

function read_data(typeData, labelsInfo, imageSize, path)
    x = zeros(size(labelsInfo, 1), imageSize)   #Intialize x matrix
    for (index, idImage) in enumerate(labelsInfo[:ID])    
        nameFile = "$(path)/$(typeData)Resized/$(idImage).Bmp"
        img = imread(nameFile)
        temp = convert(Image{Gray}, img)
        #temp = float32sc(img)
        #if ndims(temp) == 3       #Convert color images to gray images by taking the average of the color scales. 
        #    temp = mean(temp.data, 1)
        #end
        x[index, :] = reshape(temp, 1, imageSize)    #Transform image matrix to a vector and store it in data matrix 
     end 
     return x
end

read_data (generic function with 1 method)

In [8]:
imageSize = 400      # 20 x 20 pixel
labelsInfoTrain = readtable("$(wd)/trainLabels.csv")
xTrain = read_data("train", labelsInfoTrain, imageSize, wd)     #Read training matrix
labelsInfoTest = readtable("$(wd)/sampleSubmission.csv")
xTest = read_data("test", labelsInfoTest, imageSize, wd)

#Get only first character of string (convert from string to character).
#Apply the function to each element of the column "Class"
yTrain = map(x -> x[1], labelsInfoTrain[:Class])

#Convert from character to integer
yTrain = int(yTrain)

#Train random forest with 20 for number of features chosen at each random split,
#50 for number of trees, and 1.0 for ratio of subsampling.
model = build_forest(yTrain, xTrain, 20, 50, 1.0)
predTest = apply_forest(model, xTest)  #Get predictions for test data
labelsInfoTest[:Class] = char(predTest)  #Convert integer predictions to character
#writetable("$(path)/juliaSubmission.csv", labelsInfoTest, separator=',', header=true)  #Save predictions
accuracy = nfoldCV_forest(yTrain, xTrain, 20, 50, 4, 1.0);
println("4 fold accuracy: $(mean(accuracy))")


LoadError: LoadError: MethodError: `getindex` has no method matching getindex(::DataFrames.DataFrame, ::ASCIIString)
Closest candidates are:
  getindex(::DataFrames.DataFrame, !Matched::Real, !Matched::Union{Real,Symbol})
  getindex{T<:Union{Real,Symbol}}(::DataFrames.DataFrame, !Matched::Real, !Matched::AbstractArray{T<:Union{Real,Symbol},1})
  getindex(::DataFrames.DataFrame, !Matched::Real, !Matched::Colon)
  ...
while loading In[8], in expression starting on line 4

In [9]:
run(`ls -lart`)

total 24
drwxrwxr-x. 3 rmadmin rmadmin 4096 May 30 12:45 ..
drwxr-xr-x. 2 rmadmin rmadmin 4096 May 30 12:46 .ipynb_checkpoints
-rw-------. 1 rmadmin rmadmin 3223 May 30 14:00 nohup.out
-rw-rw-r--. 1 rmadmin rmadmin 5762 May 30 14:00 Julia-Kaggle0.ipynb
drwxrwxrwx. 3 rmadmin rmadmin 4096 May 30 14:00 .


Santander - XGBoost - https://www.kaggle.com/aeithne/santander-customer-satisfaction/palmares/code

In [ ]:
using DataFrames, BinDeps, XGBoost
now = time()
Patterns = readtable("/media/u01/analytics/scoring/k/Santander/train.csv")
println("Start:", time() - now);
Test = readtable("/media/u01/analytics/scoring/k/Santander/test.csv")
idList = Test[:ID]

println("Training = ", size(Patterns))
println("Test = ", size(Test))
Patterns[Patterns[:var3] .== -999999, :var3] = 2;

trainx, trainy = size(Patterns[:,1:370])
testx, testy = size(Test[:,1:370])

for i in 1:trainx
    Patterns[:var38][i] = log(Patterns[:var38][i])
end    
for i in 1:testx
    Test[:var38][i] = log(Test[:var38][i])
end

#showcols(Patterns)
xg_trainx = sparse(Array{Float64}(Patterns[2:370]))
xg_trainy = Array{Float64}(Patterns[:TARGET])
xg_testx = sparse(Array{Float64}(Test[2:370]))
xg_train = DMatrix(xg_trainx, label = xg_trainy)
xg_test = DMatrix(xg_testx)
#Patterns2 = readtable("./train_patterns.csv")
num_round = 560
max_depth = 5

dtrain = xg_train
dtest = xg_test

bst = xgboost(dtrain, num_round, eta = 0.0202048, booster = "gbtree", colsample_bytree    = 0.701, subsample           = 0.6815, metrics=["auc", "error"], max_depth = max_depth)
preds = XGBoost.predict(bst, xg_testx)
#firstload = false;
for v in 1:length(preds)
    nv = Test[:num_var33][v]+Test[:saldo_medio_var33_ult3][v]+Test[:saldo_medio_var44_hace2][v]+Test[:saldo_medio_var44_hace3][v]+
    Test[:saldo_medio_var33_ult1][v]+Test[:saldo_medio_var44_ult1][v]
    if nv > 0
        preds[v] = 0
    end
    if Test[:var15][v] < 23
        preds[v] = 0
    elseif (Test[:var15][v] > 102)
        preds[v] = 0
    end
    if Test[:saldo_medio_var5_hace2][v] > 160000
        preds[v] = 0
    end
    if Test[:saldo_var33][v] > 0
        preds[v] = 0
    end
    if Test[:var38][v] > log(3988596)
        preds[v] = 0
    end
    if Test[:var21][v] > 7500
        preds[v] = 0
    end
    if Test[:num_var30][v] > 9
        preds[v] = 0
    end
    if Test[:num_var13_0][v] > 6
        preds[v] = 0
    end
    if Test[:num_var33_0][v] > 0
        preds[v] = 0
    end
    if Test[:imp_ent_var16_ult1][v] > 51003
        preds[v] = 0
    end
    if Test[:imp_op_var39_comer_ult3][v] > 13184
        preds[v] = 0
    end
    if Test[:saldo_medio_var5_ult3][v] > 108251
        preds[v] = 0
    end
    if Test[:num_var37_0][v] > 45
        preds[v] = 0
    end
    if Test[:saldo_var5][v] > 137615
        preds[v] = 0
        end
    if Test[:saldo_var8][v] > 60099
        preds[v] = 0
    end
    if (Test[:var15][v]+Test[:num_var45_hace3][v]+Test[:num_var45_ult3][v]+Test[:var36][v]) <= 24
        preds[v] = 0
    end
    if Test[:saldo_var14][v] > 19053.78
        preds[v] = 0
    end
    if Test[:saldo_var17][v] > 288188.97
        preds[v] = 0
    end
    if Test[:saldo_var26][v] > 10381.29
        preds[v] = 0
    end
    if Test[:num_var13_largo_0][v] > 3
        preds[v] = 0
    end
    if Test[:imp_op_var40_comer_ult1][v] > 3639.87
        preds[v] = 0
    end
    
    # if (Test[:var15][v] <= 22)
        # preds[v] = 0
    # 
    
    # if Test[:saldo_var5][v] > 137615
        # preds[v] = 0
    # end
    
    # if Test[:saldo_medio_var5_hace2][v] > 160000
        # preds[v] = 0
    # end
    
    if Test[:var3][v] in [16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,38,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,68,69,71,72,73,74,76,77,78,79,81,82,84,85,86,87,88,89,90,91,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,110,112,115,117,118,119,120,121,122,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,141,143,144,145,146,147,148,149,150,151,152,153,154,156,157,158,159,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,204,205,207,208,209,210,211,213,215,216,217,218,219,220,223,225,228,229,231,235,238]
       preds[v] = 0
    end
    
    # if Test[:var33][v] > 0
        # preds[v] = 0
    # end
    # if Test[:var36][v] == 0
        # preds[v] = 0
    # end
    # if Test[:var36][v] > 3988596
        # preds[v] = 0
    # end
    # if Test[:var21][v] > 7500
        # preds[v] = 0
    # end
    if preds[v] < 0
        preds[v] = 0
    elseif preds[v] > 1
        preds[v] = 1
    end
    preds[v] = sqrt(preds[v])
end


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

submission = DataFrame(ID = idList, TARGET = preds)
writetable("juliaBenchmark.csv", submission)

<B> Rossmann-store-sales - XGBoost - https://www.kaggle.com/wacaxx/rossmann-store-sales/julia-xgboost-starter-code/code</B>

In [ ]:
using BinDeps, DataFrames, XGBoost
train = readtable("/media/u01/analytics/scoring/k/Ross/train.csv")
test = readtable("/media/u01/analytics/scoring/k/Ross/test.csv")
store = readtable("/media/u01/analytics/scoring/k/Ross/store.csv")
sample_submission = readtable("/media/u01/analytics/scoring/k/Ross/sample_submission.csv")
train = train[train[:Open] .== 1, :]   #Reduce the training set to ignore the closed stores
train = join(train, store, on = :Store, kind = :inner)  #Tables inner join
test = join(test, store, on = :Store, kind = :inner)
tesdIdx = test[:Id]  #Test Ids & closed stores indexes 
closedStoreIdx = find(test[:Open] .== 0)
#Remove NAs from competition distance data
train[isna(train[:CompetitionDistance]), :CompetitionDistance] = 20000
test[isna(test[:CompetitionDistance]), :CompetitionDistance] = 20000
#accumulatedNAInfo = colwise(isna, train)
#numberOfNAsPerColumn = map((x) -> sum(x) / length(x), accumulatedNAInfo)

#Date transformation to "Date" object
train[:Date] = Date(train[:Date], "y-m-d")
test[:Date] = Date(test[:Date], "y-m-d")

#Make new date related features
train[:year] = year(train[:Date])
train[:month] = month(train[:Date])
train[:day] = day(train[:Date])
test[:year] = year(test[:Date])
test[:month] = month(test[:Date])
test[:day] = day(test[:Date])

#Define Categorical and Numeric Columns
colummTypes = map(x -> string(x), eltypes(train))
categoricalColumns = names(train)[colummTypes .== "UTF8String"]
categoricalColumns = categoricalColumns[1:3] #Exclude promo interval
numericalColumns = [:DayOfWeek, :Promo, :SchoolHoliday, :CompetitionDistance, :Promo2, :year, :month, :day]

#Helper Function - This functions counts the occurences of a category and maps it back to the data provided
function categorical2frequency(dataVector, vectorDict)
  newVector = [vectorDict[dataPoint] for dataPoint in dataVector]
  return newVector
end

#Categorical Columns to Frequency (Counts)
for singleCol in categoricalColumns
  #singleCol = categoricalColumns[1]
  #countmap is similar to the function "table()" in R, the only difference is that countmap() returns a dictionary
  vectorDict = countmap(train[singleCol])
  #Apply counts to factors
  train[singleCol] = categorical2frequency(train[singleCol], vectorDict)
  test[singleCol] = categorical2frequency(test[singleCol], vectorDict)
  #Print progress
  print(string(singleCol) * " Column Processed")
end

#Define target
costTrainingLog = convert(Array{Float32}, train[:Sales])

#Transform data to XGBoost matrices
trainArray = convert(Array{Float32},  train[:, vcat(numericalColumns, categoricalColumns)])
testArray = convert(Array{Float32}, test[:, vcat(numericalColumns, categoricalColumns)])
dtrain = DMatrix(trainArray, label = costTrainingLog)
dtest = DMatrix(testArray)

#XGBoost training
num_round = 250
param = ["eta" => 0.2, "max_depth" => 20, "objective" => "reg:linear", "silent" => 1]
XGBoostModel = xgboost(dtrain, num_round, param = param)

preds = predict(XGBoostModel, dtest)    #Predictions using test data
preds[closedStoreIdx] = 0    #Round to zero closed stores
sampleSubmission = DataFrame(Id = tesdIdx, Sales = preds)
#writetable("juliaBenchmark.csv", sampleSubmission)   #Write Results

<B> Cern - RandomForest -  https://www.kaggle.com/gomiero/flavours-of-physics/random-forest-in-julia </B>

In [ ]:
# /media/u01/analytics/scoring/k/cern
println("spwan 8 process to speed up train...")
addprocs(8)
using DataFrames, DecisionTree

# Agreement and correlation conditions
const ks_cutoff  = 0.09
const cvm_cutoff = 0.002

function the_roc(p_labels::Array{Bool, 1}, p_data::Array{Float64, 1}, p_weights::Array{Float64, 1})
    local score_index_ordered = sortperm(p_data, rev=true)
    local labels_bool = p_labels[score_index_ordered]
    local data_ordered = p_data[score_index_ordered]
    local weights_ordered = p_weights[score_index_ordered]
    # Find threshold
    local distinct_value_indices = find(convert(Array{Bool, 1}, abs(diff(data_ordered)) .> 1.0e-08))
    local threshold_idxs = [distinct_value_indices; length(labels_bool)]
    # Accumulate true positives
    local tps = copy(weights_ordered)
    tps[find(!labels_bool)] = 0.0
    tps = cumsum(tps)[threshold_idxs]
    local fps = cumsum(weights_ordered)[threshold_idxs] .- tps
    thresholds = data_ordered[threshold_idxs]
    if length(tps) == 0 || fps[1] == 0.0
        tps = [0.0; tps]
        fps = [0.0; fps]
        thresholds = [thresholds[1] + 1.0; thresholds]
    end
    if fps[end] <= 0.0
        error("No negative samples in y_true, false positive value should be meaningless")
    else
        fpr = fps ./ fps[end]
    end
    if tps[end] <= 0.0
        error("No positive samples in y_true, true positive value should be meaningless")
    else
        tpr = tps ./ tps[end]
    end
    return fpr, tpr, thresholds
end

             
# Compute roc curve ; # data_zero: 0-labeled data ; data_one:  1-labeled data
# sample_weights_zero: weights for 0-labeled data ; sample_weights_one:  weights for 1-labeled data ; return: roc curve
function _roc_curve_splitted(data_zero::Array{Float64,1}, data_one::Array{Float64,1}, sample_weights_zero::Array{Float64,1},
                             sample_weights_one::Array{Float64,1})
    local labels = convert(Array{Bool, 1}, [zeros(data_zero); ones(data_one)])
    local data_all = [data_zero; data_one]
    local weights = [sample_weights_zero; sample_weights_one]

    fpr, tpr = the_roc(labels, data_all, weights)

    return fpr, tpr
end
                    
# Compute Kolmogorov-Smirnov (ks) distance between real data predictions cdf and Monte Carlo one.
# data_prediction: array-like, real data predictions ;  mc_prediction: array-like, Monte Carlo data predictions
# weights_data: array-like, real data weights; weights_mc: array-like, Monte Carlo weights ; returns: ks value
function compute_ks(p_data_prediction::Array{Float64,1}, p_mc_prediction::Array{Float64,1}, p_weights_data::Array{Float64,1},
                    p_weights_mc::Array{Float64,1})
    @assert length(p_data_prediction) == length(p_weights_data)
    @assert length(p_mc_prediction) == length(p_weights_mc)
    @assert reduce(&, p_data_prediction[:,1] .>= 0.0) && reduce(&, p_data_prediction[:,1] .<= 1.0)
    @assert reduce(&, p_mc_prediction[:,1] .>= 0.0) && reduce(&, p_mc_prediction[:,1] .<= 1.0)
    p_weights_data = p_weights_data ./ sum(p_weights_data)
    p_weights_mc = p_weights_mc ./ sum(p_weights_mc)
    fpr, tpr = _roc_curve_splitted(p_data_prediction, p_mc_prediction, p_weights_data, p_weights_mc)
    Dnm = maximum(abs(fpr .- tpr))
    return Dnm
end


# Rolling window: take window with definite size through the array
# data: array-like; window_size: size; return: the sequence of windows
# Example: data = array(1, 2, 3, 4, 5, 6), window_size = 4  ---- Then this function return array(array(1, 2, 3, 4), array(2, 3, 4, 5), array(3, 4, 5, 6))
function _rolling_window(p_data::Array{Int64, 1}, p_window_size::Int64)
    return [ p_data[i:i+p_window_size-1] for i in 1:length(p_data)-p_window_size ]
end

function _bincount(p_data::Array{Int64, 1}; p_weights=0, p_minlength=0)
    sz = maximum(p_data) + 1
    sz = ifelse(p_minlength > sz, p_minlength, sz)
    ret = zeros(Int64, sz)
    for i in p_data
        idx = i + 1
        ret[idx] = ret[idx] + 1
    end
    return ret
end

# Compute Cramer-von Mises metric. Compared two distributions, where first is subset of second one. Assuming that second is ordered by ascending
# subindices: indices of events which will be associated with the first distribution
# total_events: count of events in the second distribution := return: cvm metric
function _cvm(subindices, total_events::Int64)
    local target_distribution = collect(1:total_events) ./ total_events
    local subarray_distribution = cumsum(_bincount(subindices, p_minlength=total_events))
    subarray_distribution ./= subarray_distribution[end]
    return mean((target_distribution .- subarray_distribution) .^ 2)
end

                     
# Computing Cramer-von Mises (cvm) metric on background events: take average of cvms calculated for each mass bin.
# In each mass bin global prediction's cdf is compared to prediction's cdf in mass bin.
#predictions: array-like, predictions
#masses: array-like, in case of Kaggle tau23mu this is reconstructed mass
#n_neighbours: count of neighbours for event to define mass bin
#step: step through sorted mass-array to define next center of bin
#returns: average cvm value
function compute_cvm(p_predictions::Array{Float64,1}, p_masses::Array{Float64,1}, n_neighbours=200, step=50)
    @assert length(p_predictions) == length(p_masses)
    # First, reorder by masses
    local predictions_by_mass = p_predictions[sortperm(p_masses)]
    # Second, replace probabilities with order of probability among other events
    local predictions_ordered = sortperm(sortperm(predictions_by_mass))
    # Now, each window forms a group, and we can compute contribution of each group to CvM
    rw = _rolling_window(predictions_ordered, n_neighbours)
    cvms = Array(Float64, 1)
    for window in rw[1:end:step]
        push!(cvms, _cvm(window, length(predictions_ordered)))
    end
    return mean(cvms)
end

# ----- START ------

train = readtable("/media/u01/analytics/scoring/k/cern/training.csv", quotemark=Char[])
#features = convert(Array, train[:,[:LifeTime, :FlightDistance, :pt, :isolationa]])
features = convert(Array{Float64, 2}, train[:,2:31])
println("Features $(names(train[:,2:25]))")
labels = convert(Array{Float64,1}, train[:signal])
model = build_forest(labels, features, 22, 50, 1.0)   # Random Forest
#show(STDOUT, model)

# Agreement Test
check_agreement = readtable("/media/u01/analytics/scoring/k/cern/check_agreement.csv", quotemark=Char[])
features = convert(Array{Float64, 2}, check_agreement[:,2:31])
agreement_probs = apply_forest(model, features)
ks = compute_ks(agreement_probs[find(check_agreement[:signal] .==  0)], agreement_probs[find(check_agreement[:signal] .==  1)],
                check_agreement[find(check_agreement[:signal] .==  0),:weight].data, check_agreement[find(check_agreement[:signal] .==  1),:weight].data
               )
if ks < ks_cutoff
    println("The model passed the agreement test with $ks < $ks_cutoff")
else
    println("The model failed the agreement test with $ks >= $ks_cutoff")
    #exit(1)
end

# Correlation Test
check_correlation = readtable("/media/u01/analytics/scoring/k/cern/check_correlation.csv", quotemark=Char[])
features = convert(Array{Float64, 2}, check_correlation[:,2:31])
correlation_probs = apply_forest(model, features)
cvm = compute_cvm(correlation_probs, check_correlation[:mass].data)
if cvm < cvm_cutoff
    println("The model passed the correlation test with $cvm < $cvm_cutoff.")
else
    println("The model failed the correlation test with $cvm >= $cvm_cutoff.")
    #exit(2)
end

# Make predictions
test = readtable("/media/u01/analytics/scoring/k/cern/test.csv")
features = convert(Array, test[:,2:31])
predictions = apply_forest(model, features)
test[:prediction] = predictions
#writetable("jsubmission.csv", test[:,[:id,:prediction]], separator=',', header=true)
                    